$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle} $$
$$ \newcommand{\bra}[1]{\left\langle{#1}\right|} $$

# Hamiltonian Simulation
The dynamics of many simple quantum systems, with time-independent Hamiltonian H, can be found in the Schrödinger equation

$$
i\hbar \frac{d}{dt}\ket{\psi} = H\ket{\psi}
$$

with the general solution

$$
\ket{\psi(t)} = e^{-\frac{i}{\hbar}Ht}\ket{\psi(0)}
$$

We can write this in terms of a unitary operator

$$
    U = e^{-\frac{i}{\hbar}Ht}
$$

Since $\ket{\psi}$ will be an eigenvector of $H$ with eigenvalue $E$, we have 

$$
    U\ket{\psi_k} = e^{-\frac{i}{\hbar}E_kt}\ket{\psi_k}
$$

Clearly there should be a way to implement this on a quantum computer, as the eigenvalue is just a phase factor. However we encounter some problems quite early on, usually the Hamiltonian can be written in terms of individual sums (e.g. onebody and twobody part)

$$
H = \sum_k H_K
$$

Which gives

$$
    U = e^{-\frac{i}{\hbar}Ht} = e^{-\frac{i}{\hbar}\sum_k H_kt}
$$

We would like to express this in terms of products, rather than sums. This way the operator can be written as the product of some number of other operators and thus the quantum computation could be done instead of the infinite sum that is the exponent. The Baker-Campbell-Hausdorff expansion gives such an expression, but comes with an infite sum of commutators, and in most cases $[H_k,H_{k'}]\neq 0$. Luckily we can truncate the expression, hopefully without loosing too much information.

$$
    U = e^{-\frac{i}{\hbar}Ht} \approx \left(\prod_k e^{-\frac{i}{\hbar}\frac{H_kt}{\rho}}\right)^\rho + O\left(\frac{t^2}{\rho^2}\right) \approx \prod_k U_k
$$ 

where $\rho$ is the trotter number. By letting $\Delta t = \frac{t}{\rho}$ we can make a good approximation, that gets better the longer it acts. Next we want to implement this as a quantum algorithm. Assume the Hamiltonian to be on second quantized form we have

$$
H = \sum_{pq} h_{pq} a_p^\dagger a_q + \sum_{pqrs} v_{pqrs} a_p^\dagger a_q^\dagger a_s a_r
$$

This can be written in terms of Pauli operators using the Jordan-Wigner transformation or the Braviy-Kitaev transformation. The Jordan-Wigner transformation yields

$$
    a_i^\dagger = \left( \prod_{k=1}^{i-1} \sigma_z^k \right)\sigma_+^i, \quad a_i = \left( \prod_{k=1}^{i-1} \sigma_z^k \right)\sigma_-^i
$$

without having to change basis. The Braviy-Kitaev transformation is dependent on changing basis, but in turns it can be implemented using less gates than Jordan-Wigner. Lastly we need a way to implement the exponent

$$
e^{-\frac{i}{\hbar}Ht}
$$

Firstly we take notice of the relation

$$
e^{-\frac{i}{\hbar}Ot}\ket{\Phi} = \left(\cos(t)I - i \sin(t)O\right)\ket{\Phi} = \left(\cos(t)I - i \sin(t)\lambda_O \right)\ket{\Phi} = e^{-\frac{i}{\hbar}\lambda_Ot}\ket{\Phi}
$$

given some operator $O$ with eigenvalue $\lambda_O$. Next we see the how a transformation has an affect on this state

$$
U^\dagger e^{-\frac{i}{\hbar}Ot}U = U^\dagger \left(\cos(t)I - i\sin(t)O \right) U =  \left(\cos(t)U^\dagger I U  - i\sin(t)U^\dagger O U \right) = \left(\cos(t)I - i\sin(t)U^\dagger O U \right) = e^{-iU^\dagger O U t}
$$

Decomposing our Hamiltonian so that $H = U^\dagger O U$ in a complex exponent thus have a simple way of being implemented given above. Since this will be composed of the pauli operators and ladder operators given above, where

$$
\sigma_+ = \frac{1}{2}(\sigma_x + i\sigma_y), \quad \sigma_- = \frac{1}{2}(\sigma_x - i\sigma_y)
$$

and introduce the rotation operator

$$
R_n(\theta) = cos\left(\frac{\theta}{2}\right)I - i\ sin\left(\frac{\theta}{2}\right)(n_x\sigma_x + n_y\sigma_y + n_z\sigma_z)
$$

with the following relations

$$
\sigma_x = R_y(\frac{\pi}{2})\sigma_zR_y(-\frac{\pi}{2}), \quad \sigma_x = -R_z(\frac{\pi}{2})\sigma_yR_z(-\frac{\pi}{2})
\\
\sigma_y = R_z(\frac{\pi}{2})\sigma_xR_z(-\frac{\pi}{2}), \quad \sigma_y = -R_x(\frac{\pi}{2})\sigma_zR_x(-\frac{\pi}{2})
\\
 \sigma_x = H\sigma_zH, \quad \sigma_y = R_z(\frac{\pi}{2})H\sigma_z H R_z(-\frac{\pi}{2})
$$

where $H$ is the hadamard gate. 

Next to implement, e.g. the two qubit case, $e^{-i\sigma_z \otimes \sigma_z t}$ we can use the circuit below with an aditional ancilla qubit:

In [1]:
import qiskit as qk
import numpy as np

n = 2
t = 1

qr = qk.QuantumRegister(n+1)
qc = qk.QuantumCircuit(qr)
qc.cx(qr[0],qr[-1])
qc.cx(qr[1],qr[-1])
qc.rz(t,qr[-1])
qc.cx(qr[1],qr[-1])
qc.cx(qr[0],qr[-1])
qc.draw()

Where we have utilized two simple tricks. First we know that $\sigma_z$ have eigenvalues $1$ and $-1$, so acting with multiple $\sigma_z$ gates only changes the unit sign of the eigenvalue. Next we have that the $R_z$ gate in matrix form look like

$$
\begin{equation}
    R_z(\theta) = 
    \begin{bmatrix}
        e^{-i\theta} & 0 \\
        0 & e^{i\theta} \\
    \end{bmatrix}
\end{equation}
$$

Since this only gives a phase, we can "store" the parity of the qubits in the ancilla qubit and then do the rotation, as this will have the same effect as initial problem.

However, as we should aim for more efficient implementations of all circuit elements, we see that there can be done some adjustments to this. First we recognizing that we do not actually need the ancilla qubit. By storing the parity in one of the qubits and doing the rotation on this, gives the equiavalent action.
Next we see that in the case of the two qubits being acted on by $\sigma_x$ gates we need to act with hadamard gates on each side of the $\sigma_z$. this can be shortened by recognizing that the CNOT gate is equivalent to a swap gate and a CNOT (that is, reversed order CNOT) with hadamards on each side. Implementing this in the first circuit gets rid of the initial hadamard gates and, and we are only left with hadamard gates on each side of the $R_z$ gate which is equal to $HR_xH=R_z$. So instead of acting with $2n$ hadamard gates, we have to act with $2$ hadamard gates.
For the case with $\sigma_y$ gates, we still get the same reduction of hadamard gates, but still have to act with the other rotation gates.

## Example:
# Heisenberg model

The Heiseinberg model is a one-dimensional chain of spin $\frac{1}{2}$ particles, where there are only one degree of freedom being the spin. The direction of the spins and the relationship between neighbouring spins determine the energy of the system. The Hamiltonian is given by

$$
    H = h \sum_k \sigma_k^z \pm J\sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}}
$$

As we want negative eigenvalues we must alter the Hamiltonian so that $H' = H -E_{max}$. We write this in terms of dimensionless variables, by dividing with $\pm J$

$$
    \frac{H'}{\pm J} = h' \sum_k \sigma_k^z + \sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}} - E_{max}'
$$

with $h'=\frac{h}{\pm J}$ and $E_{max}'=\frac{E_{max}}{\pm J}$.

Next we write out $\sigma$ vector dot products as

$$
\vec{\sigma_k} \cdot \vec{\sigma_{k+1}} = \sigma_k^x\sigma_{k+1}^x + \sigma_k^y\sigma_{k+1}^y + \sigma_k^z\sigma_{k+1}^z
$$

Clearly our evolution operator will have four sums.

##### First sum: $h'\sum_k \sigma_k^z$
We see that 

$$
e^{-ih'\sum_k \sigma_k^zt} = \prod_k e^{-ih'\sigma_k^zt} = \prod_k R_z(h't)
$$

##### Second sum: $\sum_k \sigma_k^x\sigma_{k+1}^x$
Next we take advantage of transformation in the exponent mentioned above so that

$$
e^{-i\sum_k \sigma_k^x\sigma_{k+1}^x t} = \prod_k U_k^\dagger e^{-i\sigma_k^z\sigma_{k+1}^z t} U_k
$$

where $e^{-i\sum_k \sigma_k^z\sigma_{k+1}^z t}$ can be implemented using an ancilla qubit as a target for CNOT-gates with the simulation qubits as controls, thus affecting/storing the parity, then apply the $R_z(t)$ rotation on the ancilla, and reverse the CNOT action. From the table above we can choose $U$ as both

$$
U = H \otimes H \quad \text{or} \quad  U = R_y(-\frac{\pi}{2})\otimes R_y(-\frac{\pi}{2})
$$


##### Third sum: $\sum_k \sigma_k^y\sigma_{k+1}^y$
Repeat the process with $U$

$$
U = R_x(-\frac{\pi}{2}) \otimes R_x(-\frac{\pi}{2})
$$


##### Fourth sum: $\sum_k \sigma_k^z\sigma_{k+1}^z$
Repeat the process with $U$

$$
U = I \otimes I
$$


##### $E_{max}$ term
Need only to apply the phase $E_{max}t$ to one of the qubits in the circuit.